# Grad Cam

In [1]:
# hide
import sys
sys.path.append("..")

In [2]:
# export
# default_exp widgets.cam
from faimed3d.all import *
from fastai.basics import *

In [3]:
# export
class HookActivation():
    def __init__(self, target_layer):
        "Initialize a Pytorch hook using `hook_activation` function."
        self.hook = target_layer.register_forward_hook(self.hook_activation) 
        
    def hook_activation(self, target_layer, activ_in, activ_out): 
        "Create a copy of the layer output activations and save in `self.stored`."
        self.stored = activ_out.detach().clone()
        
    def __enter__(self, *args): 
        return self
    
    def __exit__(self, *args): 
        self.hook.remove()

        
class HookGradient():
    def __init__(self, target_layer):
        "Initialize a Pytorch hook using `hook_gradient` function."
        self.hook = target_layer.register_backward_hook(self.hook_gradient)   
        
    def hook_gradient(self, target_layer, gradient_in, gradient_out): 
        "Create a copy of the layer output gradients and save in `self.stored`."
        self.stored = gradient_out[0].detach().clone()
        
    def __enter__(self, *args): 
        return self

    def __exit__(self, *args): 
        self.hook.remove()

In [4]:
# hide 
from torchvision.models.video import r3d_18

#hide
mrnet_data = pd.read_csv('../../mri-spa/data/train_knee.csv')
std = tensor([173.06963,184.85706,197.57706,210.63336,225.09673,241.43134,260.64816,285.0106,320.0079,386.4354,562.08795])
percs = tensor([1,10,20,30,40,50,60,70,80,90,99,])


dls = ImageDataLoaders3D.from_df(mrnet_data, '/media/..',
                                 item_tfms = ResizeCrop3D(crop_by = (0., 0.1, 0.1), resize_to = (16, 112, 112), perc_crop = True),
                                 batch_tfms = [*aug_transforms_3d(), RandomPerspective3D(112, p=0.1)],
                                 valid_col = 'is_valid',
                                 bs = 64, val_bs = 64)

learn = cnn_learner_3d(dls, 
                       resnet18_3d,  
                       model_dir='/home/bressekk/Documents/faimed3d/trashbin/', 
                       metrics = [accuracy, RocAucBinary()])
learn = learn.to_fp16()

In [ ]:
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,1.269152,0.575304,0.766667,0.549895,00:43
1,1.134585,0.727970,0.583333,0.622737,00:16
2,1.024576,0.737660,0.616667,0.576842,00:16
3,0.910839,0.612966,0.708333,0.650105,00:16


In [ ]:
from fastai.learner import Learner # for compatibility with show_docs

In [ ]:
#export
@patch
def grad_cam(learn:Learner, img, target_class=None, target_layer=None, *args, **kwargs):
    
    
    if target_layer is None: target_layer = learn.model[-2][4]
    pred = learn.predict(img)
    print('actual:',pred[0], '\t predicted:', pred[1].numpy(), '\t class probabilities:', *pred[2].numpy())
    if target_class is None: target_class = pred[1].numpy()
    # get images    
    x,  = first(learn.dls.test_dl([img]))
    if x.ndim == 4: 
        x = torch.stack((x, )*3, 1)
    x_img = TensorDicom3D(learn.dls.train.decode((x,))[0][0])
    # create cam
    with HookGradient(target_layer) as hookg:
        with HookActivation(target_layer) as hook:
            output = learn.model.eval()(x.cuda())
            act = hook.stored
        output[0, target_class].backward()
        grad = hookg.stored
        p0, p1 = output.cpu().detach()[0]
        
    w = grad[0].mean(dim=(1,2,3), keepdim=True)
    global gradcam_map

    gradcam_map = (w * act[0]).sum(0).detach().cpu()
    gradcam_map = Resize3D(x_img[0].shape)(TensorDicom3D(gradcam_map))
    # plot
    x_img[0].show(*args, **kwargs)
    gradcam_map.clamp(min=0).show(cmap='inferno', alpha = 0.5, add_to_existing = True,*args, **kwargs)
    plt.axis('off')

In [ ]:
# hide
img = TensorDicom3D.create(mrnet_data.file_names[3])
learn.grad_cam(img, nrow = 5)

In [ ]:
gradcam_map.shape